# Quantum Monte Carlo Method

You remember that tutorial about adiabatic evolution? The one in which we prepared ordered ground states of Rydberg Hamiltonians, such as the $\mathbb{Z}_2$  phase? What method did we use to achieve that goal? 

Not sure? Don't worry - we actually never mentioned its name. That's because that tutorial was all about introducing those interesting ground states themselves. In this tutorial, on the other hand, the aim is to familiarize ourselves with a specific *method* that can produce such results - the **Quantum Monte Carlo** method. In particular, it will allow us to tackle system sizes far beyond what is possible with exact diagonalization - yup, that's the method used in the previous tutorial. While ED is limited to a few tens of particles, QMC can deal with hundreds. (This means you can use it benchmark your simulations on Aquila! *Cut?*)

### Monte Carlo
Before we dive into QMC, let's take a step back and make sure we understand the gist of plain Monte Carlo. Consider the following example: You are asked to calculate a really hard integral. You have no clue how to solve it analytically. Is there another way you could approach the problem? 

You could draw the corresponding graph, add a square around it and throw darts randomly at the square. For each throw, you record whether the dart landed below or above the graph. Then you form the ratio $N_{below}/N_{above}$ and multiply it by the area of the square. Indeed, after enough throws that value will approach the true area underneath the graph.

<img alt="integral" src="./integral.png">

*Note to myself: Add in a square to image*

Of course, you might spare yourself the darts and ask a computer to generate a very large number of uniformly random points scattered across the plot, to improve the accuracy of your result. As a matter of fact, the history of the Monte Carlo method and the rise of computers are closely intertwined - the very [first MC](https://en.wikipedia.org/wiki/Monte_Carlo_method#History) was run on the ENIAC itself in 1948 to simulate neutron diffusion processes in the hydrogen bomb. It was this MC simulation that gave birth to the modern era of computational physics. 

To summarize, what's the gist of Monte Carlo? Instead of solving a problem exactly, you invoke randomness to sample from the distribution involved in the problem, in this case the $f(x)$ in $\int_0^3 f(x) dx$, until your result approximates the true solution closely enough. This begs the question of what defines *closely enough*, an issue we will examine in more detail below.

### Quantum Monte Carlo

So what about *Quantum* Monte Carlo? Firstly, let us emphasize that QMC is still a *classical* simulation, i.e. it is not run on a quantum machine such as QuEra's Aquila. It is simply called *quantum* because we aim to use the idea of MC to investigate problems from quantum physics. In other words, the space we are generating samples from is a Hilbert space of quantum-mechanical configurations. Furthermore, QMC is one of the best established methods in numerically tackling the analytically intractable integrals of quantum *many-body* physics that are beyond the reach of exact solutions. 

Typically, the integral or sum in question is the expection value of some observable $\langle A \rangle_\psi = \sum_j a_j |\langle \psi | \phi_j \rangle |^2$ such as the energy, magnetization etc. The issue we face is the *curse of dimensionality*, meaning the number of terms in this sum grows exponentially in system size. We circumvent this curse by not calculating the whole sum, but instead sampling from the probability distribution given by $|\langle \psi | \phi_j \rangle |^2$, favoring those terms that contribute signficantly to the sum, i.e. have large weights $a_j$. While doing so, it is essential that we explore the configuration space ergodically. This simply means that configurations with a small but non-zero weight should have a chance of being reached, even though this will occur less frequently than for those with large weights. 

To finish of our introduction, we should mention that QMC is an umbrella term comprising many different implementations of this same core idea, each tailored to a specific class of quantum problems. Under the hood, BloqadeQMC currently implements the SSE (Stochastic Series Expansion) method. It was [first invented](http://physics.bu.edu/~sandvik/research/ssehistory.html) by Anders Sandvik to study e.g. Heisenberg-type models and recently adapted to the Rydberg Hamiltonian by [Merali et al (2021)](https://arxiv.org/abs/2107.00766).

### Getting Started with BloqadeQMC

Let's get started in the usual way by importing the required libraries:

In [1]:
using BloqadeLattices
using BloqadeExpr
using BloqadeQMC
using Random
using Plots

SyntaxError: invalid syntax (1498300373.py, line 1)

In addition, we'll import some libraries that we can later use to check our QMC against ED results for small system sizes. 

In [ ]:
using Bloqade: rydberg_density 
using Yao: mat, ArrayReg
using LinearAlgebra
using Measurements
using Measurements: value, uncertainty

As an initial example, let's recreate the $\mathbb{Z}_2$ phase in the 1D chain which we first saw in the tutorial on *Adiabatic Evolution*. This means defining the lattice geometry as well as the Rabi drive and global detuning parameters which we'll once again feed into the Rydberg Hamiltonian. 

In [ ]:
nsites = 9
atoms = generate_sites(ChainLattice(), nsites, scale = 5.72)

In [ ]:
Ω = 2π * 4
Δ = 2π * 10
h = rydberg_h(atoms; Δ, Ω)

Now comes the first new step:

In [ ]:
h_qmc = rydberg_qmc(h)

What has happened in this step? The object h_qmc still contains all the previous information about the lattice geometry as well as the Hamiltonian parameters $\Omega$ and $\Delta$. Crucially, however, this object now also stores the probability distribution from which our algorithm will sample. Without going into all the details which can be found in [Merali et al (2021)](https://arxiv.org/abs/2107.00766), let's focus on those key elements of the SSE formalism that you will need to calculate observables from the samples. This means understanding what exactly is meant by configuration space in the SSE formalism, what samples from that space look like and what their weights are. 

### The SSE Configuration Space

Let's consider a system consisting of four Rydberg atoms. One state in the SSE configuration space of this system could look like the following,

<img alt="config" src="./SSEconfig.png">

*Note: Could this picture be improved? At least the indices? And making the H / W (weight) notation consistent*

where we will briefly define each ingredient in this probably surprising picture. 

One aspect of the picture we can directly identify is its vertical dimension: the four circles correspond to four atoms, with filled circles being excited atoms $|r\rangle$ and white circles representing the ground state $|g\rangle$.

Next, let's define what the blue, red and black boxes situated either on one horizontal line or sitting between two lines stand for. These are local Hamiltonian terms that act on either one or two atoms, respectively. For example, the latter includes the Rydberg interaction term. The interested reader may find the full definition of the local terms in section 3 of [Merali et al (2021)](https://arxiv.org/abs/2107.00766).

Next, let's try to understand the horizontal dimension of the picture: Why are there seven copies of the four atoms? This is where the *SE* in SSE, the idea of *series expansion*, comes into play. In essence, the protagonist in the mathematical formalism of SSE is the finite temperature partition function $Z = Tr(e^{-\beta H})$. Instead of calculating the trace analytically, we can first write out the Taylor series of the exponential. And yes, this expansion gives rise to an intractable sum which we can use the core idea of MC to sample from. In particular, each term in that sum will have a certain expansion order $m$. It is that number that defines the horizontal length of the picture.

Putting the pieces together, we are ready to get the gist of the probability distribution stored in the rydberg_qmc object. That distribution is given by the matrix elements defined by the local operators (boxes) sandwiched in between two configurations of the physical system (two copies of the four vertically stacked atoms). The picture as a whole is what defines one sample in the SSE configuration with its weight given by the product of the matrix elements of each operator.

Don't worry if it takes you a while to process the above information. In fact, a precise understanding is not necessary to successfully run a simulation using BloqadeQMC. We have simply included this peek into the backend in order for you to have some notion of what we mean when we refer to the *number of operators sampled* in the energy calculations later on. We simply mean the number of boxes in Figure xxx.

### Running a simulation using BloqadeQMC

Now that we have defined the configuration space, let's traverse it and generate samples from it. Importantly, there is a key difference in how these samples are generated as opposed to when we were throwing darts in the beginning. Those throws were all individually random and independent. The algorithm implemented in BloqadeQMC, on the other hand, falls within the class of Markov Chain Monte Carlo (MCMC) methods. As the name suggests, the samples are no longer independent but form a chain in which the probability of the next sample depends on the current sample. (This is the so-called Markov property.)

Before jumping into the code, let's touch on the prototypical example of how such a chain might be built: via the Metropolis-Hastings algorithm. This recipe has three main steps:
1) randomly propose a new configuration, 

2) accept the proposal according to a probability given by the ratio of the Boltzmann weights of the current and proposed configuration, 
$$P = min(\frac{e^{-E_{current}/kT}}{e^{-E_{proposed}/kT}}, 1)$$

3) repeat.

*[include a paragraph on why Metropolis-Hastings is more efficient than pure random sampling?]*

Now, let's define the parameters than govern the length of the chain, i.e. how long we let the simulation run for.

In [ ]:
EQ_MCS = 100
MCS = 100_000

As you can see, we are in fact not defining one but two parameters. Indeed, a MCMC simulation is typically split into two phases: first the equilibration phase, also referred to as the *burn-in*, followed by the sampling phase. To understand what the former means and why it is necessary, let's pause to think about how the set of samples generated by algorithms such as Metropolis-Hastings relate to the probability distributions they sample from. 

Remember that at the end of the day, we want to approximate a sum where each summand comes with a certain weight. In the section on the SSE configuration space, we outlined how these weights can be calculated. But how are they used? You guessed it: in step two of the Metropolis-Hastings flavor part of the SSE algorithm. Why are we emphasizing this point? If you think carefully, you will realize that this step 2 ensures that the *number* of times a sample appears in our set of samples represents its weight, an idea we alluded to early on in this tutorial: configurations with a larger weight should appear more frequently than those with a small one.

How are these comments related to the equilibration phase? Let us first state the fact that the mathematical theorems surrounding Markov chains guarantee that these chains eventually converge to the desired probability distribution, in the sense stated above:

$$\frac{\text{#times a sample appears in the sample set}}{\text{total # of samples}} = \text{that sample's probability weight}.$$

However, it does take some time before this statement holds. Why? We must initialize the simulation, after all, and we do this at some random point in configuration space. This might very well be in a low-weight region of the space. It will take the chain some time before it starts to draw samples from the high-weight regions. Until then, low-weight configurations will dominate the set of samples and our inferred probability distribution will be distorted. It is this initial period of distortion that we wish to "burn in".

You might well be wondering how to know what value to use for your own simulation. While there is no formula that will give you to answer, a good heuristic is plotting an observable such as the energy. You will see its value initially fluctuating and eventually stabilizing around a value which you can then assume is its equilibrium expectation value.
[insert code / plot showing this?]

Now, we're almost ready to run the simulation. You can think of the BinaryThermalState object as the initialization (rephrase). You can roughly think of M as the maximum expansion order. (Indeed, we are allowed to truncate the expansion since one can show that the weights of higher terms in the expansion fall off as a Poisson distribution. Might write this up in a manual or so.) 

*Comment on rng?*

*Comment on that this beta is enough for us to be in ground state even though we are doing finite T simulation?*

In [ ]:
M = 50
ts = BinaryThermalState(h_qmc, M)
d = Diagnostics()                   # Change API s.t. Diagnostics() is not needed?

rng = MersenneTwister(3214)

β = 0.5

Time to run the simulation!

In [ ]:
[mc_step_beta!(rng, ts, H, β, d, eq=true) for i in 1:EQ_MCS] # equilibration phase

densities_QMC = zeros(nsites)
occs = zeros(MCS, nsites)

for i in 1:MCS # Monte Carlo Steps
    mc_step_beta!(rng, ts, H, β, d, eq=false) do lsize, ts, H
        spin_prop = sample(H, ts, 1)
        occs[i, :] = ifelse.(spin_prop .== true, 1.0, 0.0)
    end
end

for jj in 1:nsites
    densities_QMC[jj] = mean(occs[:,jj])
end

Let's plot the results.

In [ ]:
using Plots: bar
    
bar(occs_QMC, label="")
xlabel!("Site number")
ylabel!("Occupation density")

<img alt="9atoms" src="./9atom_chain_Z2.png">

As expected, we see a $\mathbb{Z}_2$ pattern has emerged, just as we saw using the exact diagonalization method. So let's try an example that goes beyond what is feasible with ED. We run the same code as before, substituting the 1D chain with 9 atoms for a 2D square lattice with 100 atoms. (This should only take a minute or two to run on your laptop.)

In [ ]:
nx = ny = 10
nsites = nx*ny
atoms = generate_sites(SquareLattice(), nx, ny, scale = 6.51);

<img alt="100atoms" src="./Checkerboard_10x10.png">

### Calculating observables using BloqadeQMC

The final question we will address in this tutorial is how to calculate observables using BloqadeQMC. We'll choose to investigate the energy during a detuning sweep. Furthermore, we'll limit ourselves to a system size which ED can also handle such that we may compare the results from both methods.

Let's start by returning to the 1D chain and defining the Hamiltonian parameters. We will keep the constant Rabi drive from before, i.e. $\Omega = 4 \times 2\pi$ MHz. For the detuning, we will choose the following ramp.

In [ ]:
nsites = 9
atoms = generate_sites(ChainLattice(), nsites, scale = 5.72);

Δ_step = 15
Δ = LinRange(-2π * 9, 2π * 9, Δ_step)

Now, we only need to make two small changes to our previous MC code. First, for each value of $\Delta$ specified in this ramp, we will run a separate QMC simulation that will produce one data point in the final energy plot. Second, we need to store the number of operators contained in each sample. Yes, this is where the picture we introduced earlier comes in. This number will fluctuate from sample to sample as we build the Markov chain. It is directly returned by the mc_step_beta!() function.

*[Might make sense to blackbox the energy calculation? Or do we want to explain the formula, i.e. how n is used?]*

In [ ]:
using BinningAnalysis

for ii in 1:Δ_step
        h_ii = rydberg_h(atoms; Δ = Δ[ii], Ω)
        H = rydberg_qmc(h_ii)
        ts = BinaryThermalState(H, M)
        d = Diagnostics()
    
        [mc_step_beta!(rng, ts, H, β, d, eq=true) for i in 1:EQ_MCS] #equilibration phase
        
        ns = zeros(MCS)
    
        for i in 1:MCS # Monte Carlo Steps
            ns[i] = mc_step_beta!(rng, ts, H, β, d, eq=false)
        end

        # Binning analysis 
        energy(x) = -x / β + H.energy_shift 
        BE = LogBinner(energy.(ns))
        τ_energy = tau(BE)
        τ_energy
        ratio = 2 * τ_energy + 1
        energy_binned = measurement(mean(BE), std_error(BE)*sqrt(ratio)) 
        append!(energy_QMC_β3, energy_binned)
    end

We must address one last point when calculating expectation values using a MCMC method. As discussed, the samples are not statistically independent, instead they are correlated. This point is crucial. After all, Monte Carlo is an approximate method. We must construct error bars around the mean values generated by our simulation. Correlation of samples has the effect of reducing the effective variance, thus purporting an accuracy we cannot in fact justify. 

One method of addressing this issue is via binning analysis. For details, please see the documentation of the imported package. *[or should I give details here?]*

Finally, let's carry out the ED calculation as well and plot both results together.

In [ ]:
energy_ED = zeros(Δ_step)

for ii in 1:Δ_step
    h_ii = rydberg_h(atoms; Δ = Δ[ii], Ω)
    h_m = Matrix(mat(h_ii)) 
    energies, vecs = LinearAlgebra.eigen(h_m) 

    w = exp.(-β .* (energies .- energies[1]))
    energy_ED[ii] = sum(w .* energies) / sum(w)
end

In [ ]:
scatter(Δ/2π, energy_ED, label="ED", marker=:x)
scatter!(Δ/2π, value.(energy_QMC_β3); yerror=uncertainty.(energy_QMC_β3), label="QMC", marker=:x)
xlabel!("Δ/2π")
ylabel!("Energy")

<img alt="energy" src="./Energy_Comparison.png">

*Just say "lovely, looks good!" Or actually show some statistical calculation?*

*Also, could be nice to add in an order parameter plot, e.g. staggered mag for large system size?"